In [45]:
# coding: utf-8
# a simple MLP network
import numpy as np
import scipy.io as spio
import tensorflow as tf

hidden_size=30
output_size=3
batch_size=50
beta = 0.01 # regularization

sess = tf.Session()

In [46]:
def load_data():
    data_dir = "train_test"
    X_train = spio.loadmat(data_dir + "/" + "train_data.mat")["train_data"]
    X_test = spio.loadmat(data_dir + "/" + "test_data.mat")["test_data"]
    y_train = spio.loadmat(data_dir + "/" + "train_label.mat")["train_label"]
    y_test = spio.loadmat(data_dir + "/" + "test_label.mat")["test_label"]
    return X_train, X_test, y_train, y_test

In [47]:
def normalize(X):
    mean = X.mean(axis=1).reshape(-1, 1)
    var = X.mean(axis=1).reshape(-1, 1)
    return (X - mean) / var

In [48]:
def lb2vec(y_in):
    y = y_in.ravel()
    a = y - y.min()
    b = np.zeros((a.size, a.max()+1))
    b[np.arange(a.size),a] = 1
    return b

In [49]:
X_train, X_test, y_train, y_test = load_data()

X_train = normalize(X_train)
X_test = normalize(X_test)
# map y labels to one-hot format
y_train = lb2vec(y_train)
y_test = lb2vec(y_test)

In [50]:
# MLP model
def mlp_model(input_layer, hidden_size=hidden_size):
    W1 = tf.Variable(tf.random_normal([310, hidden_size], mean=0.0, stddev=1.0))
    b1 = tf.Variable(tf.random_normal([1, hidden_size], mean=0.0, stddev=1.0))
    hidden_layer = tf.matmul(input_layer, W1) + b1
    W2 = tf.Variable(tf.random_normal([hidden_size, output_size], mean=0.0, stddev=1.0))
    b2 = tf.Variable(tf.random_normal([1, output_size], mean=0.0, stddev=1.0))
    output_layer = tf.matmul(hidden_layer, W2) + b2
    
    regularizer = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(b2)
    return output_layer, regularizer

# pipeline
input_layer = tf.placeholder(tf.float32, shape=(None, 310))
y_label = tf.placeholder(tf.float32, shape=(None, output_size))
output_layer, regularizer = mlp_model(input_layer)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=y_label)) + beta * regularizer

# opt = tf.train.AdadeltaOptimizer(0.1)
opt = tf.train.GradientDescentOptimizer(0.05)
train_step = opt.minimize(loss)

In [51]:
# Accuracy
y_predict = tf.nn.softmax(output_layer)
correct_prediction = tf.equal(tf.argmax(y_predict, axis=1), tf.argmax(y_label, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [52]:
# sess.run(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=y_label), feed_dict={input_layer: X_train, y_label: y_train})

In [53]:
# sess.run(b1, feed_dict={input_layer: X_test, y_label: y_test})

In [54]:
# sess.run(loss, feed_dict={input_layer: X_test, y_label: y_test})

In [55]:
# Train, mini-batch processing
def train():
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(1000):
        rand_index = np.random.choice(len(X_train), size=batch_size)
        rand_x = X_train[rand_index]
        rand_y = y_train[rand_index]
        train_dict = {input_layer: rand_x, y_label: rand_y}
        test_dict = {input_layer: X_test, y_label: y_test}
        sess.run(train_step, feed_dict=train_dict)
        if (i+1)% 25 == 0:
            print('Step #' + str(i+1))
            print('Train loss = ' + str(sess.run(loss, feed_dict=train_dict)))
            print('Test loss = ' + str(sess.run(loss, feed_dict=test_dict)))
            print('Accuracy(Test) = ' + str(sess.run(accuracy, feed_dict=test_dict)))
            
    return float(sess.run(accuracy, feed_dict=test_dict))
        
    

In [56]:
loss_list = []
for iters in range(10):
    print iters
    loss_list.append(train())

0
Step #25
Train loss = 50.9536
Test loss = 49.9456
Accuracy(Test) = 0.338192
Step #50
Train loss = 47.0696
Test loss = 46.8083
Accuracy(Test) = 0.48688
Step #75
Train loss = 48.8018
Test loss = 47.6677
Accuracy(Test) = 0.682216
Step #100
Train loss = 43.6031
Test loss = 43.9269
Accuracy(Test) = 0.51312
Step #125
Train loss = 43.5325
Test loss = 44.2865
Accuracy(Test) = 0.682216
Step #150
Train loss = 41.5694
Test loss = 42.0253
Accuracy(Test) = 0.661808
Step #175
Train loss = 40.3283
Test loss = 40.6993
Accuracy(Test) = 0.661808
Step #200
Train loss = 39.0242
Test loss = 39.6966
Accuracy(Test) = 0.682216
Step #225
Train loss = 37.7192
Test loss = 38.144
Accuracy(Test) = 0.51312
Step #250
Train loss = 37.0285
Test loss = 37.5829
Accuracy(Test) = 0.682216
Step #275
Train loss = 35.8108
Test loss = 36.1991
Accuracy(Test) = 0.682216
Step #300
Train loss = 34.8824
Test loss = 35.2811
Accuracy(Test) = 0.51312
Step #325
Train loss = 34.0101
Test loss = 34.5095
Accuracy(Test) = 0.682216
Step 

In [57]:
print "Average accuracy on test set(hidden_size=30): ", sum(loss_list) / len(loss_list)

Average accuracy on test set(hidden_size=30):  0.69708455205


In [31]:
print "Average accuracy on test set(hidden_size=10): ", sum(loss_list) / len(loss_list)

Average accuracy on test set(hidden_size=10):  0.726822155714


In [44]:
print "Average accuracy on test set(hidden_size=3): ", sum(loss_list) / len(loss_list)

Average accuracy on test set(hidden_size=3):  0.773760926723
